### API Portion of Code

In [18]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import json

from config import client_id
from config import client_secret
from config import redirect_uri

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,
                                               scope="user-library-read"))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Bausa  –  Marlboro Lights
1 FOURTY  –  Heimkommen
2 CRO  –  oneway
3 Bausa  –  Baron
4 Bausa  –  Tropfen
5 Gary Washington  –  Wolkenkratzer
6 The Cratez  –  Skifahren
7 CRO  –  ALLES DOPE
8 CRO  –  HÖR NICHT AUF (feat. Teesy)
9 Rammstein  –  Dicke Titten
10 Zedline  –  Polozhenie - Zedline Remix
11 Bad Religion  –  A Walk
12 Peyton Parrish  –  My Mother Told Me - Old Norse Version
13 Peyton Parrish  –  Shadow of You
14 TOOL  –  Opiate²
15 SKÁLD  –  Þat Mælti Mín Móðir
16 Hindarfjäll  –  Þat Mælti Mín Móðir
17 LocSta  –  Положение (Dior)
18 The Weeknd  –  Sacrifice
19 Marteria  –  Paradise Delay


In [23]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'
offset = 0

while True:
    response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.id,items.track.name,items.track.album.name,items.track.plays,total',
                                 additional_types=['track'])
    
    if len(response['items']) == 0:
        break
    
    pprint(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

[{'track': {'album': {'name': 'Quevedo: Bzrp Music Sessions, Vol. 52'},
            'id': '2tTmW7RDtMQtBk7m2rYeSw',
            'name': 'Quevedo: Bzrp Music Sessions, Vol. 52'}},
 {'track': {'album': {'name': "Harry's House"},
            'id': '4Dvkj6JhhA12EX05fT7y2e',
            'name': 'As It Was'}},
 {'track': {'album': {'name': 'Hounds Of Love'},
            'id': '75FEaRjZTKLhTrFGsfMUXR',
            'name': 'Running Up That Hill (A Deal With God)'}},
 {'track': {'album': {'name': 'Un Verano Sin Ti'},
            'id': '6Sq7ltF9Qa7SNFBsV5Cogx',
            'name': 'Me Porto Bonito'}},
 {'track': {'album': {'name': 'Un Verano Sin Ti'},
            'id': '1IHWl5LamUGEuP4ozKQSXZ',
            'name': 'Tití Me Preguntó'}},
 {'track': {'album': {'name': 'Glimpse of Us'},
            'id': '6xGruZOHLs39ZbVccQTuPZ',
            'name': 'Glimpse of Us'}},
 {'track': {'album': {'name': 'Un Verano Sin Ti'},
            'id': '5Eax0qFko2dh7Rl2lYs3bx',
            'name': 'Efecto'}},
 {'tra

In [11]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'

results = sp.playlist(pl_id)
print(json.dumps(results, indent=4))

{
    "collaborative": false,
    "description": "Your daily update of the most played tracks right now - Global.",
    "external_urls": {
        "spotify": "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF"
    },
    "followers": {
        "href": null,
        "total": 16922773
    },
    "href": "https://api.spotify.com/v1/playlists/37i9dQZEVXbMDoHDwVN2tF?additional_types=track",
    "id": "37i9dQZEVXbMDoHDwVN2tF",
    "images": [
        {
            "height": null,
            "url": "https://charts-images.scdn.co/assets/locale_en/regional/daily/region_global_large.jpg",
            "width": null
        }
    ],
    "name": "Top 50 - Global",
    "owner": {
        "display_name": "Spotify",
        "external_urls": {
            "spotify": "https://open.spotify.com/user/spotify"
        },
        "href": "https://api.spotify.com/v1/users/spotify",
        "id": "spotify",
        "type": "user",
        "uri": "spotify:user:spotify"
    },
    "primary_color": null,


In [9]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))

pl_id = 'spotify:playlist:37i9dQZEVXbMDoHDwVN2tF'
offset = 0

# load the first 100 songs
tracks = []
result = sp.playlist_items(pl_id, additional_types=['track'])
tracks.extend(result['items'])

# if playlist is larger than 100 songs, continue loading it until end
while result['next']:
    result = sp.next(result)
    tracks.extend(result['items'])

# remove all local songs
i = 0  # just for counting how many tracks are local
for item in tracks:
    if item['is_local']:
        tracks.remove(item)
        i += 1


# print result
print("Playlist length: " + str(len(tracks)) + "\nExcluding: " + str(i))

Playlist length: 50
Excluding: 0


In [29]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
result = sp.search("uri") #search query
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6dv1KSzpbj7bze54k5hCMi'},
  'href': 'https://api.spotify.com/v1/artists/6dv1KSzpbj7bze54k5hCMi',
  'id': '6dv1KSzpbj7bze54k5hCMi',
  'name': 'Wingnut Dishwashers Union',
  'type': 'artist',
  'uri': 'spotify:artist:6dv1KSzpbj7bze54k5hCMi'}]

In [30]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
result = sp.search("Kate Bush")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])

artist genres: ['art pop', 'art rock', 'baroque pop', 'new wave pop', 'permanent wave', 'piano rock', 'singer-songwriter']
album genres: []
album release-date: 1985-09-16


### CSV and Database Portion of Code

In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2

from config import username
from config import password
from config import host
from config import db_password

import time
%load_ext sql

In [3]:
spotify_file = 'C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\regional-global-weekly-2022-07-14.csv'

In [4]:
spotify_metadata = pd.read_csv(spotify_file, low_memory=False)

In [5]:
# Data is already clean
spotify_df = pd.DataFrame(spotify_metadata)
spotify_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146
1,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969
2,3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap Quevedo,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942
3,4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny Chencho Corleone,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392
4,5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156


In [6]:
# Setting the index
sp_df = spotify_df.set_index('rank')

In [7]:
sp_df

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
rank,,,,,,,,
1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146
2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969
3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap Quevedo,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942
4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny Chencho Corleone,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392
5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156
...,...,...,...,...,...,...,...,...
196,spotify:track:5PZAbR6JHYYV9xp5gv0XES,Ze Felipe MC Mari,Bandido,Som Livre,125,177,3,5603007
197,spotify:track:6Qyc6fS4DsZjB2mRW9DsQs,The Goo Goo Dolls,Iris,Warner Records,124,183,11,5592566
198,spotify:track:5QvW5lK6mkfTegpAK9JOO7,Fivio Foreign The Kid LAROI,Paris to Tokyo,RichFish Columbia,198,-1,1,5572362


In [16]:
# Create the output file (CSV).
output_data_file = "C:\\Users\\Blaine\\Documents\\College\\Bootcamp\\20) Final-Project\\cleaned_regional-global-weekly-2022-07-14.csv"
# Export the City_Data into a CSV.
sp_df.to_csv(output_data_file, index= True)

In [17]:
sp_genre_df = sp_df[["uri", "artist_names", "track_name", "source", "peak_rank", "previous_rank", "weeks_on_chart", "streams"]].copy()
sp_genre_df["genre"] = ""
sp_genre_df.head(10)

,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,genre
rank,,,,,,,,,
1,spotify:track:75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill A Deal With God 2018 Rema...,Rhino,1,1,7,46609146,
2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,2,15,44759969,
3,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,Bizarrap Quevedo,Quevedo Bzrp Music Sessions Vol 52,DALE PLAY Records,3,-1,1,38443942,
4,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny Chencho Corleone,Me Porto Bonito,Rimas Entertainment LLC,2,4,10,38042392,
5,spotify:track:6xGruZOHLs39ZbVccQTuPZ,Joji,Glimpse of Us,88rising Music Warner Records,1,3,5,34930156,
6,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Titi Me Pregunto,Rimas Entertainment LLC,5,5,10,32423071,
7,spotify:track:3k3NWokhRRkEPhCzPmV8TW,Bad Bunny Bomba Estereo,Ojitos Lindos,Rimas Entertainment LLC,3,6,10,28065267,
8,spotify:track:5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Rimas Entertainment LLC,7,8,10,26558071,
9,spotify:track:6Xom58OOXk2SoU711L2IXO,Bad Bunny,Moscow Mule,Rimas Entertainment LLC,2,9,10,23671403,


In [37]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
result = sp.search("Ramstein")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])


# len(sp_df.artist_names)

artist genres: ['alternative metal', 'german metal', 'industrial', 'industrial metal', 'industrial rock', 'neue deutsche harte']
album genres: []
album release-date: 1997-08-25


In [45]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri,))
                                               
for index, row in sp_df.iterrows():
    artist = row["artist_names"]

    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    print("artist genres:", artist["genres"])

# The below still needs work to add the genres to the dataframe.  Above does iterate through the list.

# try:
#     sp_genre_df.loc[index, "genre"] = genre["results"][0]["genre"]
# except (IndexError):
#     print("Artist and Genre not found... Skipping.")

artist genres: ['art pop', 'art rock', 'baroque pop', 'new wave pop', 'permanent wave', 'piano rock', 'singer-songwriter']
artist genres: ['pop']
artist genres: ['argentine hip hop', 'pop venezolano', 'trap argentino']
artist genres: ['latin', 'reggaeton', 'trap latino']
artist genres: ['viral pop']
artist genres: ['latin', 'reggaeton', 'trap latino']
artist genres: ['latin', 'reggaeton', 'trap latino']
artist genres: ['latin', 'reggaeton', 'trap latino']
artist genres: ['latin', 'reggaeton', 'trap latino']
artist genres: ['dance pop', 'pop', 'viral pop']
artist genres: ['gauze pop', 'indietronica', 'shiver pop']
artist genres: ['latin', 'reggaeton', 'reggaeton colombiano']
artist genres: ['dance pop', 'escape room', 'minnesota hip hop', 'pop', 'trap queen']
artist genres: ['colombian pop', 'dance pop', 'latin', 'latin pop', 'pop']
artist genres: ['dance pop', 'piano rock', 'pop', 'pop rock']
artist genres: ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap']
artist ge

In [7]:
db_string = f"postgresql+psycopg2://{username}:{password}@{host}/wustlfinalproject"

In [8]:
engine = create_engine(db_string)

In [9]:
# Created connection based on engine
connection = engine

In [ ]:
# This cell is broken--need further tweaks.  Data set is already up on RDS
#sp_df.to_sql('weekly_global_streams', con = engine, schema="public", if_exists="replace", index=False)

OperationalError: (psycopg2.OperationalError) could not translate host name "hp)erGa7A+L@wustlfinalproject.ck7zp9lo8lvk.us-east-2.rds.amazonaws.com" to address: Unknown server error

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [8]:
len(sp_df.columns)

8

In [12]:
len(sp_df.uri)

200